# 很简单的RNN模型，没有GPU跑得我天荒地老，随便找个模型结构直接跑了。。。

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
%matplotlib inline

from keras.models import Sequential, Model,load_model
from keras.layers import Bidirectional,Convolution1D,MaxPool1D,Flatten, TimeDistributed,RepeatVector
from keras.layers import SimpleRNN, GRU, LSTM, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D,SpatialDropout1D
from keras.layers import Dense, Activation, Reshape, Dropout, BatchNormalization, Input,concatenate
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers
from keras import backend as K
from tensorflow import set_random_seed
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler

set_random_seed(15)

Using TensorFlow backend.


In [3]:
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')


## 编码 

In [4]:
apis = list(set(list(train_data.api.unique())+list(test_data.api.unique())))
enc = LabelEncoder().fit(apis)
train_data['enc'] = enc.transform(train_data.api)
test_data['enc'] = enc.transform(test_data.api)


In [5]:
tr = train_data.groupby('file_id').enc.apply(list).reset_index()
te = test_data.groupby('file_id').enc.apply(list).reset_index()


In [6]:
label = train_data.groupby(['file_id'])['label'].agg('first').reset_index()

In [7]:
from keras.utils.np_utils import to_categorical

categorical_labels = to_categorical(label.label, num_classes=None)


In [9]:
df_train = pd.read_csv('feature/df_train_v4.csv')
df_test = pd.read_csv('feature/df_test_v4.csv')

df_train = pd.concat([df_train, pd.get_dummies(df_train.most_common_classes, prefix='most_common_classes')], axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_test.most_common_classes, prefix='most_common_classes')], axis=1)

df_train.drop('most_common_classes', axis=1, inplace=True)
df_test.drop('most_common_classes', axis=1, inplace=True)

cols = [c for c in df_train.columns if c not in ['file_id', 'label']]

df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

scaler = MinMaxScaler().fit(df_train[cols])
mm_train = scaler.transform(df_train[cols])

mm_test = scaler.transform(df_test[cols])

In [10]:
# 由于时间及内存关系只取整个进程的前5000个api
data = tr.enc.values

data = pad_sequences(data, 5000)

In [27]:
## 测试集数据
te_data = te.enc.values

te_data = pad_sequences(te_data, 7000)

## 划分验证集 8-2分

In [11]:
X_train, X_val, y_train, y_val = train_test_split(data, categorical_labels, test_size=0.2, random_state=42)

In [12]:
num_train, num_val, y_train, y_val = train_test_split(mm_train, categorical_labels, test_size=0.2, random_state=42)

## RNN模型

In [17]:
def get_model():
    models = []
    inp = Input(shape=(5000, ))
    x = Embedding(max_enc+1, 50)(inp)
    x = Bidirectional(GRU(80。))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    models.append(conc)
    
    num_inp = Input(shape=(len(cols),) )
    x_num = Dense(256, activation="relu")(num_inp)
    x_num = Dense(64, activation="relu")(x_num)
    models.append(x_num)
    
    all_inp = concatenate(models)
    outp = Dense(6, activation="softmax")(all_inp)
    
    model = Model(inputs=[inp, num_inp], outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')

    return model


In [18]:
model = get_model()

model.fit([X_train, num_train], y_train,
          batch_size=256,
          epochs=20,
          validation_data=([X_val, num_val], y_val),
          verbose=1,
          callbacks=[
              EarlyStopping(monitor='val_loss', patience=1, verbose=1, mode='min'),
              ModelCheckpoint('/tmp/lstm_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1), 

              ]
         )

Train on 93299 samples, validate on 23325 samples
Epoch 1/20
93299/93299 [==============================] - 9783s 105ms/step - loss: 0.1486 - val_loss: 0.0683

Epoch 00001: val_loss improved from inf to 0.06827, saving model to /tmp/lstm_model.h5
Epoch 2/20
93299/93299 [==============================] - 9774s 105ms/step - loss: 0.0489 - val_loss: 0.0439

Epoch 00002: val_loss improved from 0.06827 to 0.04392, saving model to /tmp/lstm_model.h5
Epoch 3/20
93299/93299 [==============================] - 9758s 105ms/step - loss: 0.0328 - val_loss: 0.0345

Epoch 00003: val_loss improved from 0.04392 to 0.03450, saving model to /tmp/lstm_model.h5
Epoch 4/20
93299/93299 [==============================] - 9764s 105ms/step - loss: 0.0257 - val_loss: 0.0303

Epoch 00004: val_loss improved from 0.03450 to 0.03026, saving model to /tmp/lstm_model.h5
Epoch 5/20
93299/93299 [==============================] - 9763s 105ms/step - loss: 0.0216 - val_loss: 0.0279

Epoch 00005: val_loss improved from 0.03

In [19]:
model.load_weights('/tmp/lstm_model.h5')

In [20]:
y_pred = model.predict([X_val, num_val])

In [ ]:
### 计算loss

In [23]:
log_loss(y_val, y_pred)

0.025828115257083836

In [28]:
y_sub = model.predict([te_data, mm_test])

## 储存提交文件

In [29]:
sub = pd.read_csv('input/3rd_security_submit_sample.csv')

sub.loc[:, ['prob0','prob1','prob2','prob3','prob4','prob5']] = y_sub

sub = sub.round(7)

sub['sum'] = sub[['prob0','prob1','prob2','prob3','prob4','prob5']].sum(1)

sub['prob0'] = sub['prob0'] / sub['sum']
sub['prob1'] = sub['prob1'] / sub['sum']
sub['prob2'] = sub['prob2'] / sub['sum']
sub['prob3'] = sub['prob3'] / sub['sum']
sub['prob4'] = sub['prob4'] / sub['sum']
sub['prob5'] = sub['prob5'] / sub['sum']

sub.head()

sub[(sub[['prob0','prob1','prob2','prob3','prob4','prob5']].sum(1) - 1) >= 1e-6]

,file_id,prob0,prob1,prob2,prob3,prob4,prob5,sum


In [30]:
sub[['file_id', 'prob0','prob1','prob2','prob3','prob4','prob5']].to_csv('rnn_02582.csv', index=False)

In [33]:
np.save('rnn_y_pred_02582.npy', y_pred)